<a href="https://colab.research.google.com/github/eightTT/Tokenizer/blob/main/Tokenizer_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

this step test the converting text to raw byte\
this is the naive way to convert human language to language that machine can understand\
in the most naive way, we can continue with this and build a 'vocabulary', but\
this way has 2 big drawbacks: cost too much memory + bad for sematic prediction\
so\
we do smth like BytePairEncoding: encode text by Pair so we can shrink the qualitymemory cost and still keep the semantic quality\
so\
we train our Tokenizer on BPE algo. and this step is Separate from LLM training. this step is called Pre-processing stuff.


In [ ]:
# source = "https://en.wikipedia.org/wiki/Symposium_(Plato)"
text = "In the Symposium, Eros is recognized both as erotic lover and as a phenomenon capable of inspiring courage, valor, great deeds and works, and vanquishing man's natural fear of death. It is seen as transcending its earthly origins and attaining spiritual heights. The extraordinary elevation of the concept of love raises a question of whether some of the most extreme extents of meaning might be intended as humor or farce. Eros is almost always translated as \"love,\" and the English word has its own varieties and ambiguities that provide additional challenges to the effort to understand the Eros of ancient Athens."
tokens = text.encode("utf-8") #raw bytes
tokens = list(map(int, tokens)) #map integer value for bytes
print('---')
print(text)
print(len(text))
print('---')
print(tokens)
print(len(tokens))

---
In the Symposium, Eros is recognized both as erotic lover and as a phenomenon capable of inspiring courage, valor, great deeds and works, and vanquishing man's natural fear of death. It is seen as transcending its earthly origins and attaining spiritual heights. The extraordinary elevation of the concept of love raises a question of whether some of the most extreme extents of meaning might be intended as humor or farce. Eros is almost always translated as "love," and the English word has its own varieties and ambiguities that provide additional challenges to the effort to understand the Eros of ancient Athens.
617
---
[73, 110, 32, 116, 104, 101, 32, 83, 121, 109, 112, 111, 115, 105, 117, 109, 44, 32, 69, 114, 111, 115, 32, 105, 115, 32, 114, 101, 99, 111, 103, 110, 105, 122, 101, 100, 32, 98, 111, 116, 104, 32, 97, 115, 32, 101, 114, 111, 116, 105, 99, 32, 108, 111, 118, 101, 114, 32, 97, 110, 100, 32, 97, 115, 32, 97, 32, 112, 104, 101, 110, 111, 109, 101, 110, 111, 110, 32, 99, 

Byte Pair Encoding (BPE)

In [ ]:
def get_common_pair(ids:list):
  '''
  the function that find the most common pair in the text corpus,
  read the list of ids,
  create (a dictionary of) pair of byte,
  count to keep tract of total number (not neccesary),
  return dictionary
  '''
  counts = {}
  for pair in zip(ids, ids[1:]):
    counts[pair] = counts.get(pair, 0) + 1
  return counts

stats = get_common_pair(tokens)
# print(stats)
# print(sorted(( (v,k) for k, v in stats.items()), reverse=True))

In [ ]:
chr(115),chr(32),chr(97),chr(110)

('s', ' ', 'a', 'n')

In [ ]:
top_pair = max(stats, key=stats.get)
top_pair

(115, 32)

the initial tokens have the ids from 0-255\
for new tokens (the pair), assign id = 256
and so on

In [ ]:
def map_new_token_with_new_id(ids, pair, idx):
  '''
  get the list of id (ids)
  the maximum pair from the ids (pair)
  repalce by a new index (idx)
  '''
  new_ids = []
  i = 0
  while i < len(ids):
    if i < len(ids)-1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      new_ids.append(idx)
      i+=2
    else:
      new_ids.append(ids[i])
      i+=1
  return new_ids

token2 = map_new_token_with_new_id(tokens, top_pair, 256)
print(token2)
print(len(token2))


[73, 110, 32, 116, 104, 101, 32, 83, 121, 109, 112, 111, 115, 105, 117, 109, 44, 32, 69, 114, 111, 256, 105, 256, 114, 101, 99, 111, 103, 110, 105, 122, 101, 100, 32, 98, 111, 116, 104, 32, 97, 256, 101, 114, 111, 116, 105, 99, 32, 108, 111, 118, 101, 114, 32, 97, 110, 100, 32, 97, 256, 97, 32, 112, 104, 101, 110, 111, 109, 101, 110, 111, 110, 32, 99, 97, 112, 97, 98, 108, 101, 32, 111, 102, 32, 105, 110, 115, 112, 105, 114, 105, 110, 103, 32, 99, 111, 117, 114, 97, 103, 101, 44, 32, 118, 97, 108, 111, 114, 44, 32, 103, 114, 101, 97, 116, 32, 100, 101, 101, 100, 256, 97, 110, 100, 32, 119, 111, 114, 107, 115, 44, 32, 97, 110, 100, 32, 118, 97, 110, 113, 117, 105, 115, 104, 105, 110, 103, 32, 109, 97, 110, 39, 256, 110, 97, 116, 117, 114, 97, 108, 32, 102, 101, 97, 114, 32, 111, 102, 32, 100, 101, 97, 116, 104, 46, 32, 73, 116, 32, 105, 256, 115, 101, 101, 110, 32, 97, 256, 116, 114, 97, 110, 115, 99, 101, 110, 100, 105, 110, 103, 32, 105, 116, 256, 101, 97, 114, 116, 104, 108, 121, 32,

the more we merge, the bigger the vocablary is, shorter sequences\
tune this to get the best vocabulary size (token size) \


In [ ]:
vocab_size = 276 # the desired final vocab size : we preset
num_merges = vocab_size - 256
ids = list(tokens) # we dont destroy the original list

merges = {} #(int1, int2) -> int3
# merge dict is maintain child1(int1), child2(int2) maping to a new tokens(int3)
i = 0
while i <= num_merges:
  stats = get_common_pair(ids)
  top_pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f"merge pair {top_pair} into new id {idx}")
  ids = map_new_token_with_new_id(ids, top_pair, idx)
  merges[top_pair] = idx
  i+=1

merge pair (115, 32) into new id 256
merge pair (101, 32) into new id 257
merge pair (97, 110) into new id 258
merge pair (116, 104) into new id 259
merge pair (100, 32) into new id 260
merge pair (105, 110) into new id 261
merge pair (101, 110) into new id 262
merge pair (116, 32) into new id 263
merge pair (111, 102) into new id 264
merge pair (264, 32) into new id 265
merge pair (111, 114) into new id 266
merge pair (258, 260) into new id 267
merge pair (97, 108) into new id 268
merge pair (259, 257) into new id 269
merge pair (97, 256) into new id 270
merge pair (116, 105) into new id 271
merge pair (110, 32) into new id 272
merge pair (114, 111) into new id 273
merge pair (261, 103) into new id 274
merge pair (274, 32) into new id 275
merge pair (97, 114) into new id 276


In [ ]:
#statistic check
print("ori tokens lenght:", len(tokens))
print("new tokens lenght:", len(ids))
print(f"compression ratio: {len(tokens)/len(ids):.2f}X")

ori tokens lenght: 617
new tokens lenght: 436
compression ratio: 1.42X


here come to the stage of **training** Tokenizer. \
we find the best size of vocabluary -> which will be store on Disk for later LLM use to learn \
we want to make sure it cover not just English \
and also code \
and maybe pixel in the future \

Decoding
Givem a sequence of integers in range [0, a], what are the text ?

In [ ]:
vocab = {idx: bytes([idx]) for idx in range(256)} # init the raw byte dictionary

for (p0, p1), idx in merges.items():         # mapping for the new token id in 'merges' to the byte object
  vocab[idx] = vocab[p0] + vocab[p1]
  # print(vocab[idx])
print('vocab: ', vocab)  # new extended vocab

# def decode(ids:list, vocab:dict):
#   '''
#   decode the sequence of integers to human text
#   using vocabulary that created before
#   output: list of string
#   '''
#   decoded_list = []
#   print('tokens:',ids)
#   for i in range(len(ids)):
#     token = vocab[ids[i]].decode("utf-8", errors = 'replace')
#     decoded_list.append(token)
#   text = ''.join(decoded_list)
#   return text

def decode(ids:list):
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors = 'replace')
  return text

a = decode(ids)
# b = decode2(ids)
print('decoded output:',a)
# print('decoded output:',b)

vocab:  {0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V', 87: b'W', 88: b'X', 89: b'Y', 90: b'Z', 9

In [ ]:
def encode(text:str):
  '''
  encode the human text to sequence of integers
  integer is map to byte object, from vocabulary
  output: list of integers
  '''
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_common_pair(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float('inf')))
    if pair not in merges:
      break
      print('Nothing else can be merge!')
    idx = merges[pair]
    tokens = map_new_token_with_new_id(tokens, pair, idx)
  return tokens
a = encode(text)
print(len(a))
encode('Hello worlds!')

436


[72, 101, 108, 108, 111, 32, 119, 266, 108, 100, 115, 33]

In [ ]:
print(decode(encode('Hello worlds!')))

Hello worlds!
